<a href="https://colab.research.google.com/github/demichie/CorsoIntroduzionePython/blob/main/lezione7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lezione 7: Manipolazione Dati Avanzata e Analisi Statistica**

Nella lezione precedente abbiamo iniziato a esplorare la libreria **Pandas**, imparando a caricare dati tabulari, a ispezionare i `DataFrame` con comandi come `.head()` e `.info()`, e a eseguire le prime operazioni di pulizia e selezione.

Oggi approfondiremo la nostra conoscenza di Pandas, passando da semplici selezioni a tecniche di manipolazione e analisi aggregata che sono al centro del lavoro di ogni data scientist. Inoltre, introdurremo **SciPy**, un'altra colonna portante dell'ecosistema scientifico di Python, per quantificare le relazioni tra i nostri dati.

### **Obiettivi dettagliati della Lezione 7:**
*   Svolgere e discutere l'esercizio per casa, consolidando le competenze della lezione precedente.
*   Imparare a **gestire i dati mancanti (`NaN`)** in modo efficace.
*   Padroneggiare una delle funzionalità più potenti di Pandas: il raggruppamento dati con **`.groupby()`**.
*   Eseguire **analisi aggregate** per calcolare statistiche sui gruppi (es. la composizione media per tipo di roccia).
*   Imparare a **ordinare** i dati in base ai valori con `.sort_values()`.
*   Discutere l'importanza del **campionamento** per un'analisi visiva chiara.
*   Quantificare i trend nei dati imparando a calcolare una **regressione lineare semplice** con **SciPy**.

## **Parte 1: Ripasso e Preparazione dei Dati**

Iniziamo la lezione svolgendo insieme l'esercizio per casa assegnato alla fine della Lezione 6. Questo ci permetterà di ripassare l'intero flusso di lavoro di base e di preparare il dataset che useremo per gli argomenti più avanzati di oggi.

**Testo dell'Esercizio:**
*   **Dataset:** "Geochemical Variations in Igneous Rocks - Mining" (`cristianminas/geochemical-variations-in-igneous-rocks-mining`).
*   **Compiti:**
    1.  Scaricare e caricare il dataset.
    2.  Ispezionare e pulire i nomi delle colonne.
    3.  Fare un riepilogo statisctico delle colonne numeriche.
    4.  Filtrare i campioni di `'ANDESITE'`.
    5.  Analizzare il contenuto di `sio2` per le andesiti.
    6.  Creare un istogramma della distribuzione di `sio2`.

### **1. Setup e Caricamento Dati**
Iniziamo importando le librerie necessarie (Pandas per i dati, NumPy per il calcolo numerico, Matplotlib per i grafici) e scarichiamo il dataset direttamente da Kaggle. Una volta ottenuto il percorso del file, carichiamo il CSV in un DataFrame.

In [ ]:
# Importiamo tutte le librerie necessarie per l'esercizio
import kagglehub
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

print("--- 1. Download e Caricamento ---")
# Scarichiamo il dataset
dataset_path_igneous = kagglehub.dataset_download("cristianminas/geochemical-variations-in-igneous-rocks-mining")
# Identifichiamo il file CSV
file_to_load_igneous = 'Data1.csv'
full_path_igneous = os.path.join(dataset_path_igneous, file_to_load_igneous)
# Carichiamo i dati
df_igneous = pd.read_csv(full_path_igneous)
print(f"File '{file_to_load_igneous}' caricato con successo.\n")

### **2. Pulizia dei Nomi delle Colonne**
Spesso i dataset grezzi hanno nomi di colonne con caratteri speciali (come `*`, `(`, `)`) o formattazioni miste che rendono difficile la scrittura del codice. Qui standardizziamo i nomi rendendoli tutti minuscoli e rimuovendo i caratteri indesiderati, per poi visualizzare le prime righe.

In [ ]:
print("--- 2. Ispezione e Pulizia ---")
print("Nomi originali:", df_igneous.columns.tolist())
# Puliamo i nomi delle colonne (minuscolo, replace di caratteri speciali)
df_igneous.columns = [col.lower().replace('*', '').replace('(', '').replace(')', '') for col in df_igneous.columns]
print("Nomi puliti:", df_igneous.columns.tolist())
print("\n")

display(df_igneous.head(5))

### **3. Panoramica Statistica**
Utilizziamo il metodo `.describe()` per ottenere un riepilogo immediato delle variabili numeriche. Questo ci permette di osservare la media, la deviazione standard, i minimi e i massimi, aiutandoci a capire il range di composizione chimica delle rocce nel dataset.

In [ ]:
print("--- 3. Riepilogo statistico delle colonne numeriche ---")
# display() formatta bene anche l'output di .describe()
display(df_igneous.describe())

### **4. Filtraggio dei Dati**
Ora selezioniamo solo i campioni di interesse. Utilizziamo il metodo `.str.contains()` sulla colonna `rock_name` per isolare tutte le righe che contengono la parola "ANDESITE". L'opzione `case=False` ci assicura di non perdere dati a causa di differenze tra maiuscole e minuscole.

In [ ]:
print("--- 4. Filtraggio ---")
# Filtriamo per 'ANDESITE'. Usiamo .str.contains() per essere robusti a eventuali spazi extra.
andesites = df_igneous[df_igneous['rock_name'].str.contains('ANDESITE', na=False, case=False)]
print(f"Trovati {len(andesites)} campioni di andesite.\n")

### **5. Analisi Specifica**
Ora che abbiamo isolato il DataFrame `andesites`, possiamo analizzarne le proprietà chimiche. Calcoliamo la concentrazione media di Silice (`sio2n`), che è il parametro principale per la classificazione delle rocce ignee.

In [ ]:
print("--- 5. Analisi ---")
sio2_andesites = andesites['sio2n']
print(f"Contenuto medio di SiO2 nelle andesiti: {sio2_andesites.mean():.2f} %\n")

### **6. Visualizzazione della Distribuzione**
Infine, creiamo un istogramma per visualizzare come varia il contenuto di silice all'interno dei campioni di andesite. Aggiungiamo una linea verticale rossa per indicare la media calcolata in precedenza, offrendo un riferimento visivo immediato rispetto alla dispersione dei dati.

In [ ]:
print("--- 6. Visualizzazione ---")
plt.figure(figsize=(10, 6))
# Usiamo .dropna() qui per evitare che eventuali NaN nella colonna sio2n causino errori nel plot
plt.hist(sio2_andesites.dropna(), bins=40, edgecolor='k')
plt.title("Distribuzione di SiO₂ nei campioni di Andesite")
plt.xlabel("SiO₂ (%)")
plt.ylabel("Frequenza")
media_sio2 = sio2_andesites.mean()
plt.axvline(media_sio2, color='red', linestyle='--', label=f'Media: {media_sio2:.2f}%')
plt.legend()
plt.grid(axis='y', alpha=0.7)
plt.show()

### **Gestire i Dati "Sporchi": i Valori Mancanti (`NaN`)**

Come abbiamo visto ispezionando sia il dataset dei vulcani che questo nuovo dataset geochimico, è molto raro che i dati del mondo reale siano completi. Spesso, per svariati motivi (errori di misurazione, dati non raccolti, problemi di fusione di tabelle), alcune celle di una tabella sono vuote.

Pandas rappresenta questi valori mancanti con un marcatore speciale, `NaN` (che sta per **"Not a Number"**), che eredita da NumPy.

**Perché i `NaN` sono un problema?**
La maggior parte delle operazioni matematiche che coinvolgono un `NaN` restituisce `NaN` come risultato (es. `5 + NaN = NaN`). Questo comportamento "si propaga" e può invalidare intere analisi statistiche o causare errori nei modelli di machine learning. È quindi essenziale identificare e gestire questi valori mancanti.

**Come identificare i `NaN`**
Abbiamo già visto il metodo `.info()` per un riepilogo. Un modo più diretto è usare `.isna()` (o il suo alias `.isnull()`):
*   `df.isna()`: Restituisce un DataFrame delle stesse dimensioni dell'originale, ma riempito di valori booleani (`True` dove c'è un `NaN`, `False` altrimenti).
*   `df.isna().sum()`: Un comando potentissimo. Concatena `.sum()` al metodo precedente per contare il numero di `True` (e quindi di `NaN`) in ogni colonna.


In [ ]:
# Useremo il DataFrame 'df_igneous' caricato nell'esercizio precedente.

print("--- Identificazione dei Valori Mancanti ---")
# Contiamo quanti NaN ci sono in ogni colonna
missing_values = df_igneous.isna().sum()

print("Numero di valori mancanti per colonna:")
# Mostriamo solo le colonne che hanno effettivamente valori mancanti
print(missing_values[missing_values > 0])

In [ ]:
# --- Gestione dei Valori Mancanti con .dropna() ---
print("\n--- Rimozione dei Valori Mancanti ---")
print(f"Numero di righe originali nel DataFrame: {len(df_igneous)}")

**Come gestire i `NaN`**
Esistono strategie complesse per modificare i valori mancanti (es. sostituirli con la media), ma la strategia più semplice e diretta è **eliminarli**.
*   **`.dropna()`**: Questo metodo rimuove le righe (o colonne) che contengono valori `NaN`.
    *   **Argomento `subset`**: Per default, `.dropna()` elimina una riga se contiene *anche un solo* `NaN` in una qualsiasi colonna. Questo può essere troppo drastico. Possiamo usare `subset=['colonna1', 'colonna2']` per specificare che vogliamo eliminare una riga solo se il `NaN` è presente in una di quelle colonne specifiche, che sono cruciali per la nostra analisi.

In [ ]:
# Per la nostra analisi, le colonne degli ossidi principali sono fondamentali.
# Creiamo un nuovo DataFrame 'df_clean' eliminando solo le righe
# che hanno NaN nelle colonne di SiO2, MgO o rock_name.
colonne_chiave = ['sio2n', 'mgon', 'rock_name']
#df_clean = df_igneous.dropna(subset=colonne_chiave)
df_clean = df_igneous.dropna(subset=colonne_chiave).copy()

print(f"Numero di righe dopo aver rimosso i NaN dalle colonne chiave: {len(df_clean)}")


In [ ]:
# Verifichiamo che non ci siano più NaN in quelle colonne specifiche
print("\nVerifica dei valori mancanti nel DataFrame pulito (colonne chiave):")
print(df_clean[colonne_chiave].isna().sum())

In [ ]:
# Nota per la lezione: d'ora in poi, per le nostre analisi, useremo questo
# DataFrame pulito, 'df_clean', per garantire la correttezza dei calcoli.

# Usiamo di nuovo .info() sul nostro DataFrame pulito per avere una visione chiara
# dei tipi di dato di ogni colonna.
print("Ispezione dei tipi di dato in 'df_clean':")
df_clean.info()

### **Investigare i Dati "Sporchi": Come Trovare i Valori Non Numerici**

L'output di `.info()` ci ha mostrato che la colonna `p2o5n` è di tipo `object`, il che indica la presenza di testo. Ma come facciamo a trovare esattamente quali sono le righe e i valori problematici in un DataFrame che potrebbe avere migliaia di righe?

Possiamo usare una tecnica di "diagnosi" molto potente:

1.  Useremo `pd.to_numeric(df['p2o5n'], errors='coerce')` per creare una **copia temporanea** della colonna in formato numerico. L'opzione `errors='coerce'` trasformerà tutti i valori che non sono numeri in `NaN`.
2.  Creeremo una maschera booleana per trovare dove sono questi `NaN` nella copia temporanea, usando il metodo `.isna()`.
3.  Useremo questa maschera per filtrare il **DataFrame originale** e visualizzare solo le righe che contengono i dati "sporchi".

In [ ]:
# Assicuriamoci che df_clean esista
if 'df_clean' in locals() and not df_clean.empty:

    # 1. Creiamo una copia temporanea della colonna in formato numerico (con NaN per gli errori)
    p2o5n_numeric_temp = pd.to_numeric(df_clean['p2o5n'], errors='coerce')

    # 2. Creiamo una maschera booleana che è True dove la conversione ha fallito (producendo NaN)
    #    e dove il valore originale non era già NaN (per isolare solo i nuovi problemi).
    valori_problematici_mask = p2o5n_numeric_temp.isna() & df_clean['p2o5n'].notna()

    # 3. Usiamo la maschera per visualizzare le righe problematiche nel DataFrame originale
    righe_problematiche = df_clean[valori_problematici_mask]

    if not righe_problematiche.empty:
        print(f"Trovate {len(righe_problematiche)} righe con valori non numerici nella colonna 'p2o5n':")
        # Mostriamo le righe e in particolare la colonna 'p2o5n'
        display(righe_problematiche[['rock_name', 'sio2n', 'p2o5n']])
    else:
        print("Nessun valore non numerico trovato (potrebbero essere già NaN).")

else:
    print("DataFrame 'df_clean' non trovato.")

### **Correzione della Colonna Problematica**

La nostra investigazione ha avuto successo: abbiamo identificato che il valore non numerico `'<0.05'` è la causa del tipo `object` della colonna `p2o5n`. Questa notazione, che probabilmente indica un valore sotto il limite di detezione, deve essere gestita prima di poter procedere con i calcoli.

La strategia più sicura e comune in questi casi è trattare questi valori come dati mancanti, in quanto non conosciamo il valore esatto.

Applicheremo ora la conversione in modo permanente al nostro DataFrame `df_clean`, usando `pd.to_numeric` con `errors='coerce'`. Questo comando sostituirà tutti i valori come `'<0.05'` con `NaN` (Not a Number), il marcatore standard di Pandas per i dati mancanti, rendendo la colonna puramente numerica.

In [ ]:
# Controlliamo se df_clean esiste prima di modificarlo
if 'df_clean' in locals() and not df_clean.empty:
    print(f"Tipo di dato di 'p2o5n' PRIMA della correzione: {df_clean['p2o5n'].dtype}")

    # Applichiamo la conversione in modo permanente alla colonna del nostro DataFrame
    # L'uso di .loc[:, 'p2o5n'] è il modo più corretto per assegnare i valori
    # ed evitare il SettingWithCopyWarning che abbiamo discusso.

    # df_clean = df_clean.copy()
    df_clean['p2o5n'] = pd.to_numeric(df_clean['p2o5n'], errors='coerce')

    print(f"Tipo di dato di 'p2o5n' DOPO la correzione: {df_clean['p2o5n'].dtype}")

    # Verifichiamo il risultato finale con .info()
    # Noteremo che 'p2o5n' è ora float64 e il suo conteggio di valori non-nulli è diminuito.
    print("\nNuovo riepilogo del DataFrame 'df_clean':")
    df_clean.info()
else:
    print("DataFrame 'df_clean' non trovato o vuoto.")

## **Parte 2: Raggruppamento e Analisi Aggregata**

Finora abbiamo filtrato i dati per selezionare sottoinsiemi di righe. Ma cosa succede se vogliamo eseguire un calcolo *su ogni categoria* presente nei nostri dati? Ad esempio, "calcola la composizione chimica media per *ogni* tipo di roccia" invece di farlo solo per le andesiti.

Eseguire questa operazione con filtri separati per ogni tipo di roccia sarebbe estremamente ripetitivo e inefficiente. Per questo tipo di analisi aggregata, Pandas offre una funzionalità potentissima: il metodo **`.groupby()`**.

### **La Logica "Split-Apply-Combine"**

Il funzionamento di `.groupby()` si basa su un processo in tre fasi, noto come **"Split-Apply-Combine"** (Dividi-Applica-Combina):

1.  **Split (Dividi):** Il DataFrame originale viene suddiviso virtualmente in gruppi, basandosi sui valori unici di una o più colonne. Ad esempio, `df.groupby('rock_name')` creerà un gruppo per 'BASALT', uno per 'ANDESITE', uno per 'RHYOLITE', e così via.

2.  **Apply (Applica):** Una funzione di aggregazione (come `.mean()`, `.sum()`, `.count()`, `.std()`) viene applicata in modo indipendente a ogni singolo gruppo. Per esempio, `.mean()` calcolerà la media di tutte le colonne numeriche per i basalti, poi per le andesiti, ecc.

3.  **Combine (Combina):** I risultati di ogni applicazione vengono raccolti e combinati in un nuovo DataFrame, dove l'indice è costituito dalle categorie usate per il raggruppamento.

Questa operazione ci permette di passare da una tabella di dati grezzi a una tabella di statistiche riassuntive con una sola, leggibile riga di codice.

In [ ]:
# Useremo il nostro DataFrame pulito, 'df_clean', per assicurarci
# che i calcoli delle medie non siano influenzati da valori mancanti.

print("--- Calcolo della composizione media per tipo di roccia ---")

# 1. SPLIT: Raggruppiamo il DataFrame per la colonna 'rock_name'.
# 2. APPLY: Calcoliamo la media (.mean()) per ogni gruppo.
# 3. COMBINE: Pandas restituisce un nuovo DataFrame con i risultati.
#    Di default, il calcolo viene eseguito solo sulle colonne numeriche.
mean_composition = df_clean.groupby('rock_name').mean()

# Visualizziamo le prime 10 righe del risultato
print("Composizione media calcolata per ogni tipo di roccia (prime 10):")
display(mean_composition.head(10))

### **Problema Comune: Categorie Duplicate a Causa di Spazi Nascosti**

Dopo aver eseguito un `groupby`, potremmo notare risultati inaspettati, come la stessa categoria (es. 'Andesite') che appare più volte. Questo è quasi sempre un sintomo di "dati sporchi" all'interno della colonna usata per il raggruppamento.

La causa più comune sono gli **spazi bianchi extra** all'inizio o alla fine delle stringhe. Per Pandas, `'Andesite'` e `'Andesite '` sono due categorie distinte.

Per verificare questo problema, possiamo usare il metodo `.unique()` su una colonna per vedere tutti i suoi valori unici. Se troviamo varianti dello stesso nome, abbiamo confermato il problema.

In [ ]:
# Ispezioniamo i valori unici nella colonna 'rock_name' del nostro DataFrame pulito
if 'df_clean' in locals() and not df_clean.empty:

    print("Valori unici nella colonna 'rock_name':")
    unique_rocks = df_clean['rock_name'].unique()

    # Stampiamo tutti i valori unici per ispezionarli visivamente
    print(unique_rocks)

    # Un modo programmatico per trovare problemi:
    # Cerchiamo i nomi delle rocce che contengono 'ANDESITE'
    andesite_variations = [rock for rock in unique_rocks if 'ANDESITE' in str(rock)]
    print("\nVariazioni trovate per 'ANDESITE':", andesite_variations)

else:
    print("DataFrame 'df_clean' non trovato.")

Dall'output, notiamo due problemi distinti:

1.  **Spazi Extra:** Alcuni nomi hanno spazi alla fine (es. vedremo `'Andesite '` o `'Gabbro  '`). Questi vengono trattati come categorie separate da `'Andesite'` e `'Gabbro'`.
2.  **Eccessiva Granularità:** Ci sono molte sottocategorie (es. `'Andesite dike'`, `'Andesite tuff'`). Per un'analisi aggregata generale, potremmo volerle raggruppare tutte sotto un'unica etichetta, "Andesite".

Risolviamo questi problemi in due passaggi.



#### **Passo 1: Pulizia degli Spazi con `.str.strip()`**

Come prima cosa, applichiamo il metodo `.str.strip()` per rimuovere tutti gli spazi bianchi all'inizio e alla fine di ogni nome di roccia. Questo accorperà le categorie che sembravano diverse solo a causa di errori di formattazione (es. `'Andesite '` diventerà uguale a `'Andesite'`).

Per misurare l'efficacia di questa operazione, useremo il metodo **`.nunique()`** (abbreviazione di *Number of Unique*).
È importante distinguere due metodi simili:
*   **`.unique()`**: Restituisce **l'elenco** dei valori distinti (es. `['Basalto', 'Andesite']`).
*   **`.nunique()`**: Restituisce **il numero** totale di valori distinti (es. `2`).

Confrontando il conteggio (`nunique`) prima e dopo la pulizia, sapremo esattamente quante categorie duplicate abbiamo eliminato.

In [ ]:
if 'df_clean' in locals():
    print(f"Nomi unici prima di .strip(): {df_clean['rock_name'].nunique()}")

    # Usiamo .loc[:, 'rock_name'] per dire a Pandas:
    # "Prendi TUTTE le righe (:) e aggiorna la colonna 'rock_name'"
    df_clean.loc[:, 'rock_name'] = df_clean['rock_name'].str.strip()

    print(f"Nomi unici dopo .strip(): {df_clean['rock_name'].nunique()}")
    # Il numero di nomi unici dovrebbe essere diminuito.

#### **Passo 2: Creare una Categoria Semplificata**

Ora che abbiamo pulito gli spazi, affrontiamo il problema della granularità. Abbiamo troppi nomi specifici (es. "High-K Basalt", "Alkali Basalt") e vogliamo raggrupparli in macro-famiglie (es. "Basaltic").

Per fare questo, useremo un costrutto molto potente e frequente in Pandas, che permette di fare una **assegnazione condizionale**.

La sintassi è:
`df.loc[CONDIZIONE_RIGHE, COLONNA_DA_MODIFICARE] = NUOVO_VALORE`

Analizziamo il comando che useremo per i Basalti:
```python
df_clean.loc[df_clean['rock_name'].str.contains('Basalt', case=False, na=False), 'rock_category'] = 'Basaltic'
```

Ecco cosa fa ogni parte:
1.  **`.loc[...]`**: È il modo corretto per selezionare dati specifici quando vogliamo modificarli (evita avvisi di memoria). Accetta due argomenti: *quali righe* e *quale colonna*.
2.  **`df_clean['rock_name'].str.contains(...)`**: Questa è la **condizione per le righe**. Cerca la sottostringa "Basalt" dentro ogni nome.
    *   **`case=False`**: Ignora le maiuscole/minuscole (trova sia "Basalt" che "basalt").
    *   **`na=False`**: Se incontra un valore mancante, lo considera `False` invece di dare errore.
3.  **`'rock_category'`**: È la **colonna** dove vogliamo scrivere il risultato.
4.  **`= 'Basaltic'`**: È il **valore** che assegniamo a quelle specifiche celle.

In parole povere: *"Cerca tutte le righe dove il nome contiene 'Basalt' e, nella colonna 'rock_category' di quelle righe, scrivi 'Basaltic'."*

In [ ]:
if 'df_clean' in locals():
    # Creiamo una nuova colonna 'rock_category' inizializzata con 'Other'
    df_clean.loc[:, 'rock_category'] = 'Other'

    # Usiamo .loc e .str.contains() per assegnare le categorie principali
    # case=False rende la ricerca insensibile a maiuscole/minuscole
    df_clean.loc[df_clean['rock_name'].str.contains('Basalt', case=False, na=False), 'rock_category'] = 'Basaltic'
    df_clean.loc[df_clean['rock_name'].str.contains('Andesite', case=False, na=False), 'rock_category'] = 'Andesitic'
    df_clean.loc[df_clean['rock_name'].str.contains('Dacite', case=False, na=False), 'rock_category'] = 'Dacitic'
    df_clean.loc[df_clean['rock_name'].str.contains('Rhyolite', case=False, na=False), 'rock_category'] = 'Rhyolitic'
    df_clean.loc[df_clean['rock_name'].str.contains('Granite', case=False, na=False), 'rock_category'] = 'Granitic'

    # Ora controlliamo il risultato con .value_counts()
    print("Conteggio delle nuove categorie semplificate:")
    display(df_clean['rock_category'].value_counts())

### **Eseguire l'Analisi Aggregata sui Dati Puliti**
Ora che abbiamo una colonna `'rock_category'` pulita e semplificata, possiamo finalmente eseguire il nostro `groupby` per calcolare la composizione media di ogni categoria principale.

In [ ]:
if 'df_clean' in locals():
    # Eseguiamo il groupby sulla nuova colonna 'rock_category'
    mean_composition_simplified = df_clean.groupby('rock_category').mean(numeric_only=True)

    print("Composizione media per categoria di roccia semplificata:")
    display(mean_composition_simplified[['sio2n', 'mgon', 'k2on']]) # Mostriamo solo alcune colonne

### **Ordinare i Dati con `.sort_values()`**

Il risultato del nostro `groupby` è un DataFrame, e come ogni DataFrame, possiamo ordinarlo per analizzarlo meglio. Il metodo `.sort_values()` ci permette di ordinare le righe in base ai valori di una o più colonne.

Gli argomenti più importanti sono:
*   `by`: il nome della colonna (o una lista di nomi) su cui basare l'ordinamento.
*   `ascending`: `True` per un ordine crescente (default), `False` per un ordine decrescente.

Ad esempio, ordiniamo i risultati precedenti per vedere quali tipi di roccia hanno, in media, il più alto contenuto di Silice.

In [ ]:
# Ordiniamo il DataFrame 'mean_composition' in base alla colonna 'sio2n'
# in ordine decrescente (dal più alto al più basso).
sorted_composition = mean_composition_simplified.sort_values(by='sio2n', ascending=False)

print("Tipi di roccia ordinati per contenuto medio di SiO₂ (primi 10):")
# Mostriamo le prime 10 rocce più ricche in silice
display(sorted_composition.head(10))

print("\nTipi di roccia ordinati per contenuto medio di MgO (primi 10):")
# E ora per MgO (ci aspettiamo un ordine inverso)
display(mean_composition_simplified.sort_values(by='mgon', ascending=False).head(10))

### **Il Campionamento dei Dati (`.sample()`)**

Il nostro dataset contiene migliaia di righe. Quando si lavora con dataset molto grandi (milioni di righe), visualizzare o elaborare tutti i dati può essere lento e portare a grafici illeggibili per la troppa sovrapposizione di punti.

Spesso è utile lavorare su un **sottoinsieme** (campione) dei dati. Pandas offre il metodo `.sample()` per estrarre righe a caso.
Proviamo a estrarre un campione casuale di 500 rocce e vediamo come sono distribuite le categorie.

In [ ]:
# Impostiamo un random_state per ottenere sempre gli stessi risultati (riproducibilità)
print("--- Campionamento Casuale Semplice (500 campioni) ---")
simple_sample = df_clean.sample(n=500, random_state=42)

# Verifichiamo la distribuzione delle categorie in questo campione
print("Conteggio per categoria nel campione casuale:")
counts_simple = simple_sample['rock_category'].value_counts()
display(counts_simple)

# Calcoliamo la percentuale rispetto al totale del campione
print("\nPercentuali nel campione casuale:")
display(counts_simple / len(simple_sample) * 100)

### **Il Problema del Bilanciamento (Bias)**

Osservando i numeri qui sopra, noterete che le categorie più comuni nel dataset originale (come 'Basaltic' o 'Andesitic') dominano anche il campione. Le categorie più rare (come 'Rhyolitic' o 'Granitic') potrebbero avere pochissimi rappresentanti o addirittura scomparire nel campione.

Se vogliamo confrontare le caratteristiche chimiche dei vari gruppi, questo è un problema: non possiamo calcolare statistiche affidabili su un gruppo se abbiamo solo 3 o 4 campioni!

Per risolvere questo problema, dobbiamo effettuare un **Campionamento Bilanciato** (o Stratificato). Invece di pescare a caso dal mucchio totale, pescheremo un numero fisso di campioni *da ogni gruppo*.

Useremo:
1.  `.groupby('rock_category')`: Per dividere i dati in gruppi.
2.  `.apply(...)`: Per applicare un campionamento su ogni singolo gruppo.

In [ ]:
print("--- Campionamento Bilanciato (Stratificato) ---")

# Definiamo quanti campioni vogliamo per ogni categoria
n_samples_per_category = 120

# Invece di usare .apply(), usiamo un approccio più esplicito e sicuro:
# 1. Creiamo una lista vuota per raccogliere i campioni
sampled_dfs = []

# 2. Cicliamo su ogni gruppo creato dal groupby
for category_name, group_df in df_clean.groupby('rock_category'):
    # Calcoliamo n (prendiamo 50 oppure tutti se sono meno di 50)
    n = min(len(group_df), n_samples_per_category)

    # Campioniamo e aggiungiamo il risultato alla lista
    sampled_dfs.append(group_df.sample(n=n, random_state=42))

# 3. Concateniamo tutti i pezzetti in un unico DataFrame finale
balanced_sample = pd.concat(sampled_dfs)

# Verifichiamo la nuova distribuzione
print("Conteggio per categoria nel campione bilanciato:")
display(balanced_sample['rock_category'].value_counts())

print(f"\nDimensione totale campione bilanciato: {len(balanced_sample)}")

## **Parte 3: Analisi di Correlazione e Regressione Lineare**

Finora abbiamo analizzato le colonne singolarmente o abbiamo aggregato i dati per categoria. Un altro compito fondamentale dell'analisi dati è capire la **relazione tra due variabili numeriche**.

### **Visualizzare le Correlazioni: i Diagrammi di Harker**

Come abbiamo visto nelle lezioni precedenti, il grafico a dispersione (`scatter plot`) è lo strumento ideale per questo. In geochimica, un'applicazione classica sono i **diagrammi di Harker**, che plottano la concentrazione di vari ossidi contro quella della silice (SiO₂).

Questi diagrammi sono potentissimi perché ci permettono di visualizzare i **trend di differenziazione magmatica**. Ad esempio, ci aspettiamo che mentre il contenuto di silice aumenta (rocce più "evolute" o felsiche), il contenuto di ossidi come il magnesio (MgO) diminuisca.

Visualizziamo questo trend con i nostri dati.

In [ ]:
# Useremo il DataFrame pulito 'df_clean'.

plt.figure(figsize=(8, 6))

# Creiamo un grafico a dispersione di SiO2 vs MgO
# Usiamo alpha=0.3 per gestire la sovrapposizione dei punti
plt.scatter(df_clean['sio2n'], df_clean['mgon'], alpha=0.3)

plt.title("Diagramma di Harker: MgO vs SiO₂")
plt.xlabel("SiO₂ (%)")
plt.ylabel("MgO (%)")
plt.grid(True)

plt.show()

# Come previsto, osserviamo una chiara correlazione negativa.

## **Visualizzazione Avanzata: Creare un Diagramma di Harker Categoriale**

Nella lezione precedente abbiamo visto come creare scatter plot e come usare colori diversi per distinguere più serie di dati. Ora possiamo combinare questa tecnica con la nostra analisi di Pandas per creare una visualizzazione molto più ricca.

Invece di plottare tutti i punti con lo stesso colore, possiamo colorare ogni punto in base alla categoria di roccia a cui appartiene (la colonna `'rock_category'` che abbiamo appena creato). Questo ci permetterà di vedere se le diverse famiglie di rocce (basaltiche, andesitiche, ecc.) si raggruppano in zone specifiche del diagramma.

**La Strategia:**
Il modo più elegante per fare questo con Matplotlib è:
1.  Ottenere la lista di tutte le categorie uniche di rocce.
2.  Creare un ciclo `for` che itera su ogni categoria.
3.  All'interno del ciclo:
    a. Selezionare un sotto-DataFrame contenente solo le rocce di quella categoria.
    b. Creare uno `scatter plot` solo per quel sotto-DataFrame, assegnandogli un colore e un'etichetta specifici.

In questo modo, Matplotlib creerà automaticamente un grafico con punti di colori diversi e una legenda corretta.

In [ ]:
# Assicuriamoci che df_clean e la colonna 'rock_category' esistano
if 'balanced_sample' in locals() and 'rock_category' in balanced_sample.columns:

    # --- Creazione di un Diagramma di Harker Categoriale ---

    # 1. Definiamo i dati da plottare
    x_axis = 'sio2n'
    y_axis = 'mgon'

    # Inizializziamo la figura
    plt.figure(figsize=(10, 8))

    # 2. Otteniamo la lista delle categorie uniche di rocce
    categories = balanced_sample['rock_category'].unique()

    # (Opzionale) Creiamo una mappa di colori per assegnare un colore a ogni categoria
    colors = plt.cm.viridis(np.linspace(0, 1, len(categories)))
    color_map = dict(zip(categories, colors))

    # 3. Cicliamo su ogni categoria per plottarla separatamente
    for category in categories:
        # Selezioniamo il sotto-DataFrame per la categoria corrente
        subset = balanced_sample[balanced_sample['rock_category'] == category]

        # Plottiamo solo i dati di questo subset
        plt.scatter(subset[x_axis], subset[y_axis],
                    label=category,
                    color=color_map[category],
                    alpha=0.6, s=50) # s=50 per la dimensione dei punti

    # Personalizziamo il grafico
    plt.title(f"Diagramma di Harker: {y_axis.upper()} vs {x_axis.upper()}")
    plt.xlabel("SiO₂ (%)")
    plt.ylabel("MgO (%)")
    plt.grid(True, linestyle='--', alpha=0.25)

    # Aggiungiamo la legenda
    plt.legend(title="Categoria di Roccia")

    plt.show()

else:
    print("DataFrame 'df_clean' o colonna 'rock_category' non trovati. Esegui le celle precedenti.")

### **Quantificare il Trend: Regressione Lineare Semplice con SciPy**

Il nostro occhio vede chiaramente una linea. Ma come possiamo descrivere questa relazione matematicamente? Vogliamo trovare l'equazione della retta (`y = mx + q`) che "meglio approssima" i nostri dati. Questo processo è chiamato **regressione lineare semplice**.

Per eseguirla, useremo la libreria **SciPy** (Scientific Python), un'altra colonna portante dell'ecosistema scientifico di Python che fornisce algoritmi per l'ottimizzazione, l'algebra lineare, l'integrazione e, appunto, la statistica.

La funzione che ci interessa è `linregress()` dal modulo `scipy.stats`.

**La funzione `linregress(x, y)`:**
*   Prende in input due sequenze di dati (le nostre colonne `x` e `y`).
*   Esegue i calcoli della regressione lineare.
*   Restituisce una tupla con cinque valori importantissimi:
    *   `slope` (pendenza): il coefficiente `m` della retta. Ci dice di quanto cambia `y` per ogni unità di `x`.
    *   `intercept` (intercetta): il valore `q` della retta. È il valore di `y` quando `x` è 0.
    *   `rvalue` (coefficiente di correlazione): un valore tra -1 e 1 che misura la forza e la direzione della relazione lineare.
    *   `pvalue`: un valore statistico che ci aiuta a capire se la correlazione trovata è significativa o dovuta al caso (un p-value basso, es. < 0.05, è buono).
    *   `stderr`: l'errore standard della stima della pendenza.

Nella prossima cella, useremo questa funzione per calcolare la retta di miglior fit e la sovrapporremo al nostro grafico.

In [ ]:
# Importiamo la funzione linregress dal modulo scipy.stats
from scipy.stats import linregress

# Assicuriamoci che il DataFrame 'df_clean' esista e non sia vuoto
if 'df_clean' in locals() and not df_clean.empty:

    # --- Esecuzione della Regressione Lineare ---

    # 1. Prepariamo i dati
    # Per evitare problemi con i NaN che potrebbero essere ancora presenti,
    # creiamo un piccolo DataFrame temporaneo con solo le due colonne di interesse
    # e rimuoviamo le righe dove almeno uno dei due valori è mancante.
    regression_data = df_clean[['sio2n', 'mgon']].dropna()

    x_data = regression_data['sio2n']
    y_data = regression_data['mgon']

    # 2. Eseguiamo la regressione lineare con linregress
    slope, intercept, r_value, p_value, std_err = linregress(x_data, y_data)

    # 3. Stampiamo i risultati in modo leggibile
    print("--- Risultati della Regressione Lineare (MgO vs SiO₂) ---")
    print(f"Pendenza (slope, m): {slope:.4f}")
    print(f"Intercetta (intercept, q): {intercept:.4f}")
    print(f"Coefficiente di correlazione (R): {r_value:.4f}")
    print(f"Coefficiente di determinazione (R-squared): {r_value**2:.4f}")
    print(f"P-value: {p_value:.3e}") # Usiamo la notazione scientifica per p-value molto piccoli

    # Interpretazione dei risultati
    print("\nInterpretazione:")
    print(f"La retta di regressione è: MgO = {slope:.2f} * SiO₂ + {intercept:.2f}")
    if p_value < 0.05:
        print("Il p-value è molto basso, indicando che la correlazione è statisticamente significativa.")
    if r_value < 0:
        print(f"Il coefficiente di correlazione R di {r_value:.2f} indica una forte correlazione negativa.")

else:
    print("DataFrame 'df_clean' non trovato. Impossibile eseguire la regressione.")

### Interpretazione dei Risultati Statistici

Vediamo ora come interpretare i parametri statistici chiave ottenuti dalla regressione lineare tra **MgO** (variabile dipendente) e **SiO₂** (variabile indipendente).

#### 1. Coefficiente di determinazione ($R^2$)
Il **Coefficiente di determinazione**, o $R^2$, misura la proporzione della varianza nella variabile dipendente che è prevedibile dalla variabile indipendente. Varia da 0 a 1 e rappresenta la "bontà di adattamento" del modello ai dati reali.
*   **Significato generale:** Un valore vicino a 1 indica che il modello spiega bene la variabilità dei dati; un valore vicino a 0 indica che il modello non spiega la variabilità.
*   **Nel nostro caso:** Il valore ottenuto di **0.6932** indica che circa il **69.3%** della variazione nel contenuto di MgO può essere spiegata dalla variazione del contenuto di SiO₂ attraverso questo modello lineare. Il restante ~30.7% della variabilità è dovuto ad altri fattori non inclusi nel modello o a varianza casuale.

#### 2. P-value
Il **P-value** viene utilizzato per verificare la significatività statistica della relazione ipotizzata. Testa l'ipotesi nulla ($H_0$) secondo cui non esiste alcuna relazione tra le due variabili (ovvero, che la pendenza della retta sia zero).
*   **Significato generale:** Solitamente, un P-value inferiore a **0.05** (5%) indica che si può rifiutare l'ipotesi nulla, confermando che esiste una relazione significativa tra le variabili.
*   **Nel nostro caso:** Il P-value di **0.000e+00** (praticamente zero) è ben al di sotto della soglia di significatività di 0.05. Ciò ci permette di affermare con altissima confidenza che la relazione osservata tra MgO e SiO₂ **è statisticamente significativa** e non è frutto del caso.

---
**Conclusione:**
I dati mostrano una relazione **significativa** (basso P-value) e **moderatamente forte** (buon $R^2$). Dato che la pendenza ($m$) e il coefficiente di correlazione ($R$) sono negativi, possiamo confermare che esiste una **correlazione inversa**: all'aumentare della silice ($SiO_2$), il contenuto di magnesio ($MgO$) diminuisce sistematicamente.

### **Visualizzare la Retta di Regressione**

Abbiamo calcolato i parametri della retta che meglio approssima i nostri dati (`slope` e `intercept`). Ora, il passo finale è visualizzare questa retta direttamente sopra i nostri dati per valutarne visivamente la bontà dell'adattamento (*goodness of fit*).

Per disegnare una linea, Matplotlib ha bisogno solo di due punti. La nostra strategia sarà:

1.  **Generare i Punti per la Retta:**
    *   Creeremo un piccolo array NumPy `x_line` contenente solo due valori: il minimo e il massimo del nostro range di SiO₂.
    *   Applicheremo l'equazione della retta (`y = mx + q`) a questi due punti per calcolare i corrispondenti valori `y_line`. Questi due punti `(x_min, y_min_calcolato)` e `(x_max, y_max_calcolato)` definiscono l'inizio e la fine della nostra retta di regressione.

2.  **Creare il Grafico:**
    *   Per prima cosa, ricreeremo lo `scatter plot` originale con tutti i nostri dati.
    *   Successivamente, sullo **stesso grafico**, useremo `plt.plot()` con i punti `x_line` e `y_line` per sovrapporre la retta di regressione.

3.  **Annotare il Grafico:**
    *   Un buon grafico scientifico non solo mostra i dati, ma riporta anche i risultati chiave dell'analisi. Aggiungeremo una casella di testo direttamente sul grafico per visualizzare l'equazione della retta e il valore di R², rendendo la figura auto-esplicativa.

In [ ]:
# Assicuriamoci che i risultati della regressione esistano
if 'slope' in locals():

    # --- Visualizzazione della Retta di Regressione ---

    # 1. Generiamo i punti della retta di regressione
    # Creiamo un array x per la linea (dai valori min e max dei nostri dati SiO2)
    x_line = np.array([x_data.min(), x_data.max()])
    # Calcoliamo i corrispondenti valori y usando l'equazione della retta
    y_line = slope * x_line + intercept

    # 2. Creiamo di nuovo lo scatter plot
    plt.figure(figsize=(10, 8))
    plt.scatter(x_data, y_data, alpha=0.3, label='Dati Originali')

    # 3. Sovrapponiamo la retta di regressione
    plt.plot(x_line, y_line, color='red', linestyle='--', linewidth=2, label='Retta di Regressione')

    # 4. Aggiungiamo le personalizzazioni
    plt.title("Diagramma di Harker con Retta di Regressione")
    plt.xlabel("SiO₂ (%)")
    plt.ylabel("MgO (%)")
    plt.grid(True, linestyle='--', alpha=0.5)

    # Aggiungiamo una casella di testo con i risultati della regressione
    # La stringa 'f-string' multilinea è molto comoda per questo
    stats_text = (
        f"y = {slope:.2f}x + {intercept:.2f}\n"
        f"$R^2$ = {r_value**2:.2f}"  # Usiamo il formato LaTeX per R^2
    )
    # 'transform=ax.transAxes' posiziona il testo in coordinate relative al grafico
    plt.gca().text(0.65, 0.85, stats_text, transform=plt.gca().transAxes,
                   fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

    plt.legend()
    plt.show()

else:
    print("Esegui prima la cella precedente per calcolare i parametri della regressione.")

### **Analisi Avanzata: Regressione Lineare per Gruppo**

Abbiamo calcolato una singola retta di regressione per l'intero dataset. Tuttavia, dal nostro grafico a dispersione categoriale, abbiamo notato che le diverse famiglie di rocce (`rock_category`) occupano zone diverse del diagramma di Harker.

Una domanda più interessante è: **ogni categoria di roccia segue un proprio trend di differenziazione?**

Per rispondere, possiamo combinare la potenza di `.groupby()` con `linregress`. La nostra strategia sarà:

1.  **Raggruppare i Dati:** Useremo `df_clean.groupby('rock_category')` per creare gruppi distinti per ogni tipo di roccia (Basaltic, Andesitic, etc.).
2.  **Iterare sui Gruppi:** Creeremo un ciclo `for` che "visita" ogni gruppo uno per uno.
3.  **Calcolare la Regressione per Ogni Gruppo:** All'interno del ciclo, eseguiremo `linregress` solo sui dati del gruppo corrente.
4.  **Visualizzare i Risultati:** Creeremo un unico grafico a dispersione con tutti i punti colorati per categoria (come abbiamo fatto prima) e poi, all'interno dello stesso ciclo, sovrapporremo la retta di regressione specifica per ogni gruppo, usando lo stesso colore.

Questo ci permetterà di confrontare visivamente i trend di differenziazione delle diverse famiglie magmatiche.

In [ ]:
# Importiamo la funzione, se non già fatto
from scipy.stats import linregress

# Assicuriamoci che df_clean e la colonna 'rock_category' esistano
if 'balanced_sample' in locals() and 'rock_category' in balanced_sample.columns:

    # --- Creazione del Grafico a Dispersione Categoriale Iniziale ---
    fig, ax = plt.subplots(figsize=(12, 9))

    categories = balanced_sample['rock_category'].unique()

    colors = plt.cm.viridis(np.linspace(0, 1, len(categories)))
    color_map = dict(zip(categories, colors))

    # Plottiamo prima tutti i punti per avere lo sfondo
    for category in categories:
        subset = balanced_sample[balanced_sample['rock_category'] == category]
        ax.scatter(subset['sio2n'], subset['mgon'],
                   label=category,
                   color=color_map[category],
                   alpha=0.4) # Alpha più basso per i punti

    # --- Calcolo e Plot della Regressione per Ogni Gruppo ---
    print("--- Risultati della Regressione per Gruppo ---")

    # Raggruppiamo per categoria di roccia
    grouped = balanced_sample.groupby('rock_category')

    for name, group in grouped:
        # Pulisci i dati per la regressione (rimuovi NaN solo per le colonne di interesse)
        regression_data = group[['sio2n', 'mgon']].dropna()

        # Esegui la regressione solo se ci sono abbastanza punti (es. > 10)
        if len(regression_data) > 10:
            x_data = regression_data['sio2n']
            y_data = regression_data['mgon']

            slope, intercept, r_value, p_value, std_err = linregress(x_data, y_data)

            # Stampa i risultati per questo gruppo
            print(f"\nGruppo: {name}")
            print(f"  R-squared: {r_value**2:.2f}, P-value: {p_value:.2e}")

            # Genera i punti per la retta di regressione
            x_line = np.array([x_data.min(), x_data.max()])
            y_line = slope * x_line + intercept

            # Sovrapponi la retta al grafico, usando lo stesso colore dei punti
            ax.plot(x_line, y_line,
                    color=color_map[name],
                    linestyle='-',
                    linewidth=2.5)

    # Personalizzazione finale del grafico
    ax.set_title("Diagramma di Harker con Rette di Regressione per Categoria")
    ax.set_xlabel("SiO₂ (%)")
    ax.set_ylabel("MgO (%)")
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.legend(title="Categoria di Roccia")

    plt.show()

else:
    print("DataFrame 'df_clean' o colonna 'rock_category' non trovati. Esegui le celle precedenti.")

### **Visualizzare l'Incertezza: Gli Intervalli di Confidenza**

Le rette che abbiamo disegnato finora sono le stime "migliori". Tuttavia, in statistica è fondamentale comunicare anche l'**incertezza** di questa stima.

Se avessimo preso un campione diverso di rocce, la retta sarebbe leggermente diversa. L'**Intervallo di Confidenza al 95%** (Confidence Interval) viene visualizzato come una banda semitrasparente attorno alla retta. Ci dice che, se ripetessimo il campionamento infinite volte, il 95% delle rette calcolate cadrebbe all'interno di quella banda.

*   **Banda stretta:** Abbiamo molti dati e la correlazione è forte (siamo sicuri della posizione della retta).
*   **Banda larga:** Abbiamo pochi dati o i dati sono molto dispersi (c'è molta incertezza sulla pendenza reale).

### **Dietro le Quinte: Calcolare l'Incertezza (Intervallo di Confidenza)**

Nel grafico che stiamo per generare, non ci limiteremo a disegnare la retta di regressione. Disegneremo anche una **banda colorata** attorno ad essa, che rappresenta l'**Intervallo di Confidenza al 95%**.

Questa bandanon **non avrà una larghezza costante**: sarà più stretta al centro e si allargherà alle estremità (a forma di clessidra o "farfallino").

#### **Perché ha questa forma?**
La regressione lineare funziona come una leva che ruota attorno al centro di massa dei dati (la media di $x$ e la media di $y$).
1.  **Al centro:** Siamo molto sicuri della posizione della retta.
2.  **Ai bordi:** Piccoli errori nella stima della pendenza si amplificano man mano che ci allontaniamo dal centro, aumentando l'incertezza.

#### **La Matematica del Codice**
Nel codice sottostante, calcoliamo questa banda "manualmente" per ogni punto $x$ della linea usando questa formula statistica per l'intervallo di confidenza $IC$:

$$ IC = \pm t \cdot s_e \cdot \sqrt{\frac{1}{n} + \frac{(x - \bar{x})^2}{\sum (x_i - \bar{x})^2}} $$

Dove i componenti principali sono:
*   **$t$ (t-student):** Un fattore statistico che dipende da quanto "sicuri" vogliamo essere (95%).
*   **$s_e$ (Standard Error):** Misura quanto i punti reali sono dispersi attorno alla retta. Se i dati sono molto sparpagliati, la banda sarà più larga.
*   **$\frac{1}{n}$:** Dipende dal numero di campioni. Più dati abbiamo, più piccolo è questo termine e più stretta è la banda.
*   **$(x - \bar{x})^2$:** Questo è il termine che crea la forma a clessidra. L'incertezza aumenta col quadrato della distanza dal valore medio ($\bar{x}$).

Useremo la funzione `plt.fill_between()` di Matplotlib per visualizzare l'area compresa tra il valore superiore e inferiore calcolati con questa formula.

In [ ]:
from scipy import stats

if 'balanced_sample' in locals():

    plt.figure(figsize=(12, 9))

    # Definiamo le categorie e i colori
    categories = balanced_sample['rock_category'].unique()
    colors = plt.cm.viridis(np.linspace(0, 1, len(categories)))
    color_map = dict(zip(categories, colors))

    for category in categories:
        subset = balanced_sample[balanced_sample['rock_category'] == category]

        # Saltiamo gruppi troppo piccoli
        if len(subset) < 10:
            continue

        x = subset['sio2n']
        y = subset['mgon']

        # 1. Scatter Plot dei punti
        plt.scatter(x, y, label=category, color=color_map[category], alpha=0.3)

        # 2. Calcoli per la regressione e l'intervallo di confidenza
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

        # Creiamo una sequenza di X per disegnare la linea e la banda
        x_line = np.linspace(x.min(), x.max(), 100)
        y_line = slope * x_line + intercept

        # --- Calcolo della Banda di Confidenza ---
        # (Statistica: Errore Standard della stima)
        # Calcoliamo i residui e la somma dei quadrati
        y_pred_data = slope * x + intercept
        residuals = y - y_pred_data
        sum_errs = np.sum(residuals**2)
        stdev = np.sqrt(sum_errs / (len(y) - 2)) # Deviazione standard dei residui

        # Calcoliamo l'intervallo per ogni punto della linea x_line
        # Formula dell'intervallo di previsione media
        ci = stdev * np.sqrt(1/len(x) + (x_line - x.mean())**2 / np.sum((x - x.mean())**2))

        # Valore t-student per il 95% di confidenza
        t_stat = stats.t.ppf(0.975, len(x) - 2)

        upper_bound = y_line + (t_stat * ci)
        lower_bound = y_line - (t_stat * ci)

        # 3. Plot della linea
        plt.plot(x_line, y_line, color=color_map[category], linewidth=2)

        # 4. Plot della banda (fill_between)
        plt.fill_between(x_line, lower_bound, upper_bound, color=color_map[category], alpha=0.15)

    plt.title("Diagramma di Harker con Bande di Confidenza (Calcolo Manuale)")
    plt.xlabel("SiO₂ (%)")
    plt.ylabel("MgO (%)")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend(title="Categoria")
    plt.show()

Fortunatamente, Python ci offre delle librerie grafiche che fanno in modo automatico tutto quello che abbiamo fatto manualmente nella cella precedente.

Per realizzare questo grafico complesso in modo semplice, possiamo infatti utilizzare **Seaborn**, una libreria di visualizzazione statistica basata su Matplotlib che automatizza il calcolo delle regressioni e degli intervalli di confidenza.

In [ ]:
import seaborn as sns

# Assicuriamoci che balanced_sample esista
if 'balanced_sample' in locals():

    # Impostiamo lo stile di Seaborn (opzionale, per renderlo più gradevole)
    sns.set_theme(style="whitegrid")

    # Usiamo lmplot (Linear Model Plot).
    # Fa tutto lui: scatter plot, regressione per gruppi, e bande di confidenza.
    g = sns.lmplot(
        data=balanced_sample,
        x='sio2n',
        y='mgon',
        hue='rock_category',  # Colora e divide le regressioni per categoria
        height=7,             # Altezza della figura
        aspect=1.3,           # Larghezza (aspetto)
        scatter_kws={'alpha': 0.5, 's': 30}, # Opzioni per i puntini (trasparenza e dimensione)
        line_kws={'linewidth': 2},           # Opzioni per le linee
        ci=95                 # Confidence Interval al 95% (default)
    )

    # Personalizziamo i titoli usando l'oggetto 'g' (FacetGrid)
    g.set_axis_labels("SiO₂ (%)", "MgO (%)")
    g.fig.suptitle("Regressione Lineare con Intervalli di Confidenza (95%)", y=1.02, fontsize=16)

    plt.show()

else:
    print("DataFrame 'balanced_sample' non trovato.")